In [1]:
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import playercareerstats, leaguegamefinder, playergamelog, boxscoreadvancedv3, teamestimatedmetrics, playerestimatedmetrics
from nba_api.stats.library.parameters import Season, SeasonType
import pandas as pd
import numpy as np
from datetime import date
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier
import statsmodels.api as sm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, GridSearchCV
import sklearn
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import MinMaxScaler

In [3]:
#Predict above/below this number
point_threshold = "27.5"
player_full_name = "Tyrese Haliburton"
player_team_abbr = "IND"
opp_team_abbrev = "BOS"
game_date = "DEC 04, 2023"

In [4]:
#Fetch player data
player_details = [p for p in players.get_players() if p["full_name"] == player_full_name][0]

career_stats = playercareerstats.PlayerCareerStats(player_id=player_details['id']).get_data_frames()[0]

seasons = career_stats['SEASON_ID']

season_data = {}
for s in seasons:
    season_data[s] = playergamelog.PlayerGameLog(player_id=player_details['id'], season=s, season_type_all_star=SeasonType.regular).get_data_frames()[0]

In [5]:
"""
For each game, we want to extract the following information for training:
1. Opponent team defensive rating
2. Player team offensive rating
3. Player offensive rating
4. Player usage rate
5. Days since last game
6. Game number
"""

'\nFor each game, we want to extract the following information for training:\n1. Opponent team defensive rating\n2. Player team offensive rating\n3. Player offensive rating\n4. Player usage rate\n5. Days since last game\n6. Game number\n'

In [6]:
#Form training arrays
X_currseason = []
X_allseasons = []
opp_def_rating_allseasons = []
opp_def_rating_currseason = []
team_off_rating_allseasons = []
team_off_rating_currseason = []
player_off_rating_allseasons = []
player_off_rating_currseason = []
player_usage_rate_allseasons = []
player_usage_rate_currseason = []
days_since_last_game_allseasons = []
days_since_last_game_currseason = []
game_numbers_allseasons = []
game_numbers_currseason = []
Y_regr_currseason = []
Y_class_currseason = []
Y_regr_allseasons = []
Y_class_allseasons = []



In [7]:
"""
Collect data. There's a different approach here. 
We are going to build two models, one predicting exact points (regression), one
predicting purely over/under point threshold. Also, we are going to train on two 
different timeframes of data: player whole career and current season only.
"""
for season in season_data.keys():
    currSeason = season_data[season]
    for ind in currSeason.index:
        #Label regressions
        Y_regr_allseasons.append(float(currSeason['PTS'][ind]))
        
        #Get defensive rating
        metrics = teamestimatedmetrics.TeamEstimatedMetrics(league_id="00", season=season, season_type = SeasonType.regular).get_data_frames()[0]
        opp_abbrev = currSeason['MATCHUP'][ind][-3:]


        for i in teams.get_teams():
            if i['abbreviation'] == opp_abbrev:
                opp_team_id = i['id']
                
        opp_def_rating_allseasons.append(metrics.loc[metrics["TEAM_ID"] == opp_team_id]["E_DEF_RATING"])

        #Get team offensive rating
        for i in teams.get_teams():
            if i['abbreviation'] == currSeason["MATCHUP"][ind][0:3]:
                plr_team_id = i['id']

        team_off_rating_allseasons.append(metrics.loc[metrics["TEAM_ID"] == plr_team_id]["E_OFF_RATING"])
        
        #Get player offensive rating
        player_metrics = playerestimatedmetrics.PlayerEstimatedMetrics(league_id="00", season=season, season_type = SeasonType.regular).get_data_frames()[0]
        player_id = player_details['id']

        player_off_rating_allseasons.append(player_metrics.loc[player_metrics["PLAYER_ID"] == player_id]["E_OFF_RATING"])

        #Get player usage rate
        player_usage_rate_allseasons.append(player_metrics.loc[player_metrics["PLAYER_ID"] == player_id]["E_USG_PCT"])
        
        #Label classifications
        if float(currSeason['PTS'][ind]) > float(point_threshold):
            Y_class_allseasons.append(1)
        else:
            Y_class_allseasons.append(0)
            
for ind in season_data["2023-24"].index:
    #Label regressions
    Y_regr_currseason.append(float(season_data["2023-24"]['PTS'][ind]))
    
    #Get team defensive ratings
    metrics = teamestimatedmetrics.TeamEstimatedMetrics(league_id="00", season="2023-24", season_type = SeasonType.regular).get_data_frames()[0]
    opp_abbrev = season_data["2023-24"]["MATCHUP"][ind][-3:]

    for i in teams.get_teams():
        if i['abbreviation'] == opp_abbrev:
            opp_team_id = i['id']

    opp_def_rating_currseason.append(metrics.loc[metrics["TEAM_ID"] == opp_team_id]["E_DEF_RATING"])

    #Get team offensive ratings
    for i in teams.get_teams():
        if i['abbreviation'] == currSeason["MATCHUP"][ind][0:3]:
            plr_team_id = i['id']

    team_off_rating_currseason.append(metrics.loc[metrics["TEAM_ID"] == plr_team_id]["E_OFF_RATING"])

    #Get player offensive rating
    player_metrics = playerestimatedmetrics.PlayerEstimatedMetrics(league_id="00", season="2023-24", season_type = SeasonType.regular).get_data_frames()[0]
    player_id = player_details['id']

    player_off_rating_currseason.append(player_metrics.loc[player_metrics["PLAYER_ID"] == player_id]["E_OFF_RATING"])

    #Get player usage rate
    player_usage_rate_currseason.append(player_metrics.loc[player_metrics["PLAYER_ID"] == player_id]["E_USG_PCT"])
    
    #Label classifications
    if float(season_data["2023-24"]['PTS'][ind]) > float(point_threshold):
        Y_class_currseason.append(1)
    else:
        Y_class_currseason.append(0)
    

In [8]:
months = {
    "JAN": 1,
    "FEB": 2,
    "MAR": 3,
    "APR": 4,
    "MAY": 5,
    "JUN": 6,
    "JUL": 7,
    "AUG": 8,
    "OCT": 10,
    "NOV": 11,
    "DEC": 12
}
def convert_human_date(d):
    month = months[d[0:3]]
    day = int(d[4:6])
    year = int(d[8:])

    return date(year, month, day)

In [9]:
def distance_between_dates(d1, d2):
    return int((d2 - d1).days)

In [10]:
#default days of rest set for the first game of the season:
first_game_rest = 7

In [11]:
#Compute days since last game
for season in season_data.keys():
    currSeason = season_data[season]
    for ind in currSeason.index:
        if (ind + 1) == len(currSeason.index):
            days_since_last_game_allseasons.append(first_game_rest)
        else:
            currGameDate = convert_human_date(str(currSeason["GAME_DATE"][ind]))
            lastGameDate = convert_human_date(str(currSeason["GAME_DATE"][ind+1]))
            days_since_last_game_allseasons.append(distance_between_dates(lastGameDate, currGameDate))
for ind in season_data["2023-24"].index:
    if (ind + 1) == len(season_data["2023-24"].index):
        days_since_last_game_currseason.append(first_game_rest)
    else:
        currGameDate = convert_human_date(str(season_data['2023-24']["GAME_DATE"][ind]))
        lastGameDate = convert_human_date(str(season_data['2023-24']["GAME_DATE"][ind+1]))
        days_since_last_game_currseason.append(distance_between_dates(lastGameDate, currGameDate))

In [12]:
#Get the game number, in terms of the number of games that player has played
for season in season_data.keys():
    for ind in season_data[season].index:
        game_numbers_allseasons.append(len(season_data[season].index) - ind)

for ind in season_data["2023-24"].index:
    game_numbers_currseason.append(len(season_data["2023-24"].index) - ind)
        

In [13]:
#default average ppg since start of season
default_first_game_ppg = 0.0

In [14]:
#Zip together all the training input data
X_allseasons = []
X_currseason = []
for i in range(len(game_numbers_allseasons)):
    tpl = (float(opp_def_rating_allseasons[i]), float(team_off_rating_allseasons[i]), float(player_off_rating_allseasons[i]), float(player_usage_rate_allseasons[i]), float(days_since_last_game_allseasons[i]), float(game_numbers_allseasons[i]))
    X_allseasons.append(tpl)

for i in range(len(game_numbers_currseason)):
    tpl = (float(opp_def_rating_currseason[i]), float(team_off_rating_currseason[i]), float(player_off_rating_currseason[i]), float(player_usage_rate_currseason[i]), float(days_since_last_game_currseason[i]), float(game_numbers_currseason[i]))
    X_currseason.append(tpl)
    

/var/folders/wt/9_6pdt8n2kz5f1cv6bqysq440000gn/T/ipykernel_3182/4020995574.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  tpl = (float(opp_def_rating_allseasons[i]), float(team_off_rating_allseasons[i]), float(player_off_rating_allseasons[i]), float(player_usage_rate_allseasons[i]), float(days_since_last_game_allseasons[i]), float(game_numbers_allseasons[i]))
/var/folders/wt/9_6pdt8n2kz5f1cv6bqysq440000gn/T/ipykernel_3182/4020995574.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  tpl = (float(opp_def_rating_currseason[i]), float(team_off_rating_currseason[i]), float(player_off_rating_currseason[i]), float(player_usage_rate_currseason[i]), float(days_since_last_game_currseason[i]), float(game_numbers_currseason[i]))


In [15]:
#Run PCA to determine which predictor variables to include, for all seasons
pca = PCA(n_components = 6)
pca.fit(X_allseasons)
print(pca.explained_variance_ratio_)

[9.17849779e-01 5.32424172e-02 1.50497608e-02 1.14535192e-02
 2.40447152e-03 5.24759352e-08]


In [16]:
#So then let's reform the data, without game number and days since last game from the data
X_allseasons = []
X_currseason = []
for i in range(len(game_numbers_allseasons)):
    tpl = (float(opp_def_rating_allseasons[i]), float(team_off_rating_allseasons[i]), float(player_off_rating_allseasons[i]), float(player_usage_rate_allseasons[i]))
    X_allseasons.append(tpl)

for i in range(len(game_numbers_currseason)):
    tpl = (float(opp_def_rating_currseason[i]), float(team_off_rating_currseason[i]), float(player_off_rating_currseason[i]), float(player_usage_rate_currseason[i]))
    X_currseason.append(tpl)
    

/var/folders/wt/9_6pdt8n2kz5f1cv6bqysq440000gn/T/ipykernel_3182/4277634942.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  tpl = (float(opp_def_rating_allseasons[i]), float(team_off_rating_allseasons[i]), float(player_off_rating_allseasons[i]), float(player_usage_rate_allseasons[i]))
/var/folders/wt/9_6pdt8n2kz5f1cv6bqysq440000gn/T/ipykernel_3182/4277634942.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  tpl = (float(opp_def_rating_currseason[i]), float(team_off_rating_currseason[i]), float(player_off_rating_currseason[i]), float(player_usage_rate_currseason[i]))


In [17]:
#Now, let's start preprocessing the data
Y_regr_allseasons = pd.DataFrame(Y_regr_allseasons)
Y_regr_currseason = pd.DataFrame(Y_regr_currseason)
Y_class_allseasons = pd.DataFrame(Y_class_allseasons)
Y_class_currseason = pd.DataFrame(Y_class_currseason)

X_allseasons = pd.DataFrame(X_allseasons)
X_currseason = pd.DataFrame(X_currseason)

In [18]:
X_regr_allseasons_train, X_regr_allseasons_test, Y_regr_allseasons_train, Y_regr_allseasons_test = train_test_split(X_allseasons, Y_regr_allseasons, test_size=.25)
X_class_allseasons_train, X_class_allseasons_test, Y_class_allseasons_train, Y_class_allseasons_test = train_test_split(X_allseasons, Y_class_allseasons, test_size=.25)
X_regr_currseason_train, X_regr_currseason_test, Y_regr_currseason_train, Y_regr_currseason_test = train_test_split(X_currseason, Y_regr_currseason, test_size=.25)
X_class_currseason_train, X_class_currseason_test, Y_class_currseason_train, Y_class_currseason_test = train_test_split(X_currseason, Y_class_currseason, test_size=.25)

In [19]:
#Let's start with the classification models. We'll check with 5-fold cross validation

In [20]:
#all seasons SVC. Has a great cross-validation score
svc = SVC()
scores = cross_val_score(svc, X_class_allseasons_train, Y_class_allseasons_train.values.ravel(), cv=5)
print(np.mean(scores))

0.9038306451612904


In [21]:
#current season SVC. Not great. 
svc = SVC()
scores = cross_val_score(svc, X_class_currseason_train, Y_class_currseason_train.values.ravel(), cv=5)
print(scores)

[0.66666667 0.66666667 0.5        0.5        0.5       ]


In [22]:
#All seasons logistic classifier. Pretty good!
log = LogisticRegression()
scores = cross_val_score(log, X_class_allseasons_train, Y_class_allseasons_train.values.ravel(), cv=5)
print(np.mean(scores))

0.8651209677419354


In [23]:
#Current season logistic classifier. Bad
log = LogisticRegression()
scores = cross_val_score(log, X_class_currseason_train, Y_class_currseason_train.values.ravel(), cv=5)
print(scores)

[0.33333333 0.33333333 0.5        0.5        0.5       ]


In [24]:
#What's happening is a lack of data problem. There have been fewer than twenty games this season for any given team. So let's use all season data now. 

In [25]:
#All season random forest classifier. Not as good as SVC or logistic classifier. 
rcf = RandomForestClassifier()
scores = cross_val_score(rcf, X_class_allseasons_train, Y_class_allseasons_train.values.ravel(), cv=5)
print(np.mean(scores))

0.8715725806451612


In [26]:
#Gradient boosting. Not as great as SVC or logistic classifier.
gbc = GradientBoostingClassifier()
scores = cross_val_score(gbc, X_class_allseasons_train, Y_class_allseasons_train.values.ravel(), cv=5)
print(np.mean(scores))

0.8522177419354838


In [27]:
#Ridge classifier. This is pretty good too. 
rc = RidgeClassifier()
scores = cross_val_score(rc, X_class_allseasons_train, Y_class_allseasons_train.values.ravel(), cv=5)
print(np.mean(scores))

0.878024193548387


In [28]:
#Now, let's try regression on allseasons.

In [29]:
#Linear regression
OLS_model = sm.OLS(Y_regr_allseasons_train, X_regr_allseasons_train, hasconst=None)
OLS_results = OLS_model.fit()
OLS_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      0   R-squared (uncentered):                   0.843
Model:                            OLS   Adj. R-squared (uncentered):              0.839
Method:                 Least Squares   F-statistic:                              204.4
Date:                Mon, 04 Dec 2023   Prob (F-statistic):                    4.52e-60
Time:                        21:04:22   Log-Likelihood:                         -533.16
No. Observations:                 156   AIC:                                      1074.
Df Residuals:                     152   BIC:                                      1087.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
0             -0.1411      0.193     -0.733      0.465      -0.522       0.239
1              1.4426      0.897      1.608      0.110      -0.330       3.215
2             -1.5011      1.070     -1.402      0.163      -3.616       0.614
3            196.7925     68.856      2.858      0.005      60.754     332.831
==============================================================================
Omnibus:                        2.266   Durbin-Watson:                   2.048
Prob(Omnibus):                  0.322   Jarque-Bera (JB):                1.820
Skew:                           0.235   Prob(JB):                        0.403
Kurtosis:                       3.243   Cond. No.                     2.21e+04
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 2.21e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [31]:
#Set up constants for weighted linear regression
y_resid = [abs(resid) for resid in OLS_results.resid]

X_resid = sm.add_constant(OLS_results.fittedvalues)

mod_resid = sm.OLS(y_resid, X_resid)
res_resid = mod_resid.fit()

mod_fv = res_resid.fittedvalues

weights = 1 / (mod_fv**2)

In [32]:
#Create WLS model
WLS_model = sm.WLS(Y_regr_allseasons_train, X_regr_allseasons_train, hasconst=None)
WLS_results = WLS_model.fit()
WLS_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 WLS Regression Results                                
=======================================================================================
Dep. Variable:                      0   R-squared (uncentered):                   0.843
Model:                            WLS   Adj. R-squared (uncentered):              0.839
Method:                 Least Squares   F-statistic:                              204.4
Date:                Mon, 04 Dec 2023   Prob (F-statistic):                    4.52e-60
Time:                        21:05:35   Log-Likelihood:                         -533.16
No. Observations:                 156   AIC:                                      1074.
Df Residuals:                     152   BIC:                                      1087.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
0             -0.1411      0.193     -0.733      0.465      -0.522       0.239
1              1.4426      0.897      1.608      0.110      -0.330       3.215
2             -1.5011      1.070     -1.402      0.163      -3.616       0.614
3            196.7925     68.856      2.858      0.005      60.754     332.831
==============================================================================
Omnibus:                        2.266   Durbin-Watson:                   2.048
Prob(Omnibus):                  0.322   Jarque-Bera (JB):                1.820
Skew:                           0.235   Prob(JB):                        0.403
Kurtosis:                       3.243   Cond. No.                     2.21e+04
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 2.21e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [38]:
#Random forest regression
rfr = RandomForestRegressor()
scores = cross_val_score(rfr, X_regr_allseasons_train, Y_regr_allseasons_train.values.ravel(), cv=5)
print(np.mean(scores))

-0.22392892976206227


In [39]:
#Gradient boosted regression
gbr = GradientBoostingRegressor()
scores = cross_val_score(gbr, X_regr_allseasons_train, Y_regr_allseasons_train, cv=5)
print(np.mean(scores))

-0.22504395478445355


/Users/nehalchigurupati/miniconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nehalchigurupati/miniconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nehalchigurupati/miniconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nehalchigurupati/miniconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d

In [ ]:
#I think the best approach then is to go with the logistic regressor. Cleanly implemented in PlayerPointModelV3.